# 🏍️ LossZero: Motorcycle Night Ride Semantic Segmentation

이 노트북은 야간 오토바이 주행 이미지를 활용하여 다중 클래스 시멘틱 세그멘테이션 모델을 훈련합니다.  
**Raw 이미지**와 **JSON 어노테이션**을 사용하여 훈련 파이프라인을 구축하며, W&B를 통해 실험을 추적합니다.

In [2]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import segmentation_models_pytorch as smp
import albumentations as A # 딥러닝 분야, 특히 세그멘테이션(사물 경계 나누기)에서 세계적으로 가장 많이 쓰이는 도구입니다.
from albumentations.pytorch import ToTensorV2
import wandb

print(f"PyTorch version: {torch.__version__}")

/Users/jamesyang/.pyenv/versions/3.12.2/envs/aipel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 2.10.0


In [6]:
def get_device():
    """
    사용 가능한 최적의 장치(CUDA, MPS, CPU)를 반환합니다.
    """
    if torch.cuda.is_available():
        return "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return "mps"
    return "cpu"

In [7]:
# ⚙️ 설정 (Configuration)
DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset") 
JSON_PATH = os.path.join(DATA_DIR, "COCO_motorcycle (pixel).json")
IMG_DIR = os.path.join(DATA_DIR, "images")

CFG = {
    "project": "LossZero",
    "base_model": "DeepLabV3Plus",
    "encoder": "resnet34",
    "img_size": (512, 512),
    "batch_size": 8,
    "lr": 1e-4,
    "epochs": 50,
    "device": get_device()
}

print(f"Data directory: {DATA_DIR}")
print(f"JSON exists: {os.path.exists(JSON_PATH)}")

Data directory: /Users/jamesyang/Projects/LossZero/data/Motorcycle Night Ride Dataset
JSON exists: True


### 🚜 Dataset 클래스 정의 (Raw + JSON ➔ Mask)
JSON에 적힌 문자열 좌표를 실시간으로 픽셀 단위 마스크로 변환하여 모델에게 전달합니다.

In [ ]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    """
    JSON의 좌표 데이터를 보고 도화지에 정답(Mask)을 그리는 함수
    """
    ann_ids = coco.getAnnIds(imgIds=img_id) #해당 사진에 달린 정답지(어노테이션) 번호를 몽땅 가져와
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    
    for ann in anns:
        cat_id = ann['category_id']
        cls_idx = id_to_idx[cat_id]
        pixel_mask = coco.annToMask(ann) # 좌표 -> 픽셀 마스크 변환
        mask[pixel_mask == 1] = cls_idx   # 해당 영역을 클래스 번호로 색칠
        
    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    """
    이미지 한 장을 읽고, 마스크를 생성하고, 증강(Augmentation)까지 수행하는 함수
    """
    # 1. 정보 및 이미지 로드
    img_info = coco.loadImgs(img_id)[0] # 원본 이미지...
    img_path = os.path.join(img_dir, img_info['file_name'])
    
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 2. 마스크 생성 (함수 호출)
    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)
    
    # 3. 데이터 증강 및 텐서 변환
    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']
    
    return image, torch.as_tensor(mask).long()

In [21]:
# 🖼️ 데이터 증강 설정
train_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),  # 512, 512
    # A.HorizontalFlip(p=0.5), #A.HorizontalFlip(p=0.5) : "거울 모드로 보여주기" 데이타 증강 2배
    #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # 사진마다 조명이 너무 밝거나 어두우면 AI가 학습하기 힘들어합니다.
    # 여기 적힌 숫자들(mean, std)은 전 세계 수백만 장의 표준 사진들의 평균값입니다. 
    # 이 필터를 통과하면 모든 사진의 픽셀 값이 유사한 범위(-1에서 1 사이 등)로 재조정되어 학습이 훨씬 빨라집니다.
    ToTensorV2() #텐서 변환
])
# 🛠️ COCO 데이터 정보 로드
coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys()) # 이 함수는 데이터셋에 들어있는 모든 이미지 사진의 고유 번호(ID)를 반환합니다.
print(img_ids)
id_to_idx = {cat_id: i for i, cat_id in enumerate(coco.getCatIds())} # 이 함수는 Category ID
print(id_to_idx)

loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
{1329681: 0, 1323885: 1,

In [22]:
# 🏗️ 모델 및 손실함수 설정
model = smp.DeepLabV3Plus(
    encoder_name=CFG['encoder'],   # resnet34
    encoder_weights="imagenet",    # 이미 공부가 좀 된 백본 사용
    in_channels=3,                 # RGB 컬러 이미지
    classes=len(coco.getCatIds())  # 분류할 사물 개수
).to(CFG['device'])                # GPU/MPS/CPU 장치로 전송

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr']) # 공부법
criterion = smp.losses.DiceLoss(mode='multiclass')             # 채점 방식(경계선 위주)

In [25]:
# 🚀 학습 시작 (순수 반복문 방식)
for epoch in range(CFG['epochs']):
    model.train()
    np.random.shuffle(img_ids) # 매번 공부 순서를 바꿉니다.
    
    epoch_loss = 0
    # 배치 크기(8장)만큼씩 끊어서 공부합니다.
    for i in range(0, len(img_ids), CFG['batch_size']):
        batch_ids = img_ids[i : i + CFG['batch_size']]
        
        # [함수 호출] 이미지와 정답지 세트 만들기
        images, masks = [], []
        for img_id in batch_ids:
            img, msk = process_single_data(coco, img_id, IMG_DIR, id_to_idx, train_transform)
            images.append(img)
            masks.append(msk)
        
        # 덩어리로 묶어서 장치(GPU 등)로 보냅니다.
        X = torch.stack(images).to(CFG['device']).float() # 입력사진...
        y = torch.stack(masks).to(CFG['device']).long() # 정답지... 좌표값이 아니라, 좌표를 바탕으로 그려진 "정답 지도(Mask)"임.
        
        # 공부 진행
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{CFG['epochs']}] Avg Loss: {epoch_loss/(len(img_ids)//CFG['batch_size']):.4f}")

Epoch [1/50] Avg Loss: 0.7035
Epoch [2/50] Avg Loss: 0.4594
Epoch [3/50] Avg Loss: 0.3421
Epoch [4/50] Avg Loss: 0.2925
Epoch [5/50] Avg Loss: 0.2475
Epoch [6/50] Avg Loss: 0.2178
Epoch [7/50] Avg Loss: 0.1939
Epoch [8/50] Avg Loss: 0.1725
Epoch [9/50] Avg Loss: 0.1535
Epoch [10/50] Avg Loss: 0.1463
Epoch [11/50] Avg Loss: 0.1311
Epoch [12/50] Avg Loss: 0.1241
Epoch [13/50] Avg Loss: 0.1158
Epoch [14/50] Avg Loss: 0.1089
Epoch [15/50] Avg Loss: 0.1062
Epoch [16/50] Avg Loss: 0.1008
Epoch [17/50] Avg Loss: 0.0949
Epoch [18/50] Avg Loss: 0.0910
Epoch [19/50] Avg Loss: 0.0897
Epoch [20/50] Avg Loss: 0.0879
Epoch [21/50] Avg Loss: 0.0839
Epoch [22/50] Avg Loss: 0.0842
Epoch [23/50] Avg Loss: 0.0829
Epoch [24/50] Avg Loss: 0.0816
Epoch [25/50] Avg Loss: 0.0772
Epoch [26/50] Avg Loss: 0.0748
Epoch [27/50] Avg Loss: 0.0756
Epoch [28/50] Avg Loss: 0.0759
Epoch [29/50] Avg Loss: 0.0720
Epoch [30/50] Avg Loss: 0.0691
Epoch [31/50] Avg Loss: 0.0678
Epoch [32/50] Avg Loss: 0.0696
Epoch [33/50] Avg